# Dynamic Web scraper

This notebook scrapes the MSU courses website. 

In [1]:
#Load selinimum and automatically install the Chrome Driver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import re

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from getpass import getpass
from bs4 import BeautifulSoup

from Login import login_to_SIS

In [2]:
# Chrome Driver
# options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(options=options)

In [2]:
# FireFox Driver
driver = webdriver.Firefox()
time.sleep(5)

In [3]:
url = "https://student.msu.edu/psc/public/EMPLOYEE/SA/c/NUI_FRAMEWORK.PT_AGSTARTPAGE_NUI.GBL?CONTEXTIDPARAMS=TEMPLATE_ID%3aPTPPNAVCOL&scname=MSU_AA_SCHEDULE_NEW0&PanelCollapsible=Y"
driver.get(url)
time.sleep(5)

body = driver.page_source
soup = BeautifulSoup(body, 'html.parser')
cells = soup.find_all("tr", class_="ps_grid-row psc_rowact")
semesters = dict()
for cell in cells: # Find each semester's ID
    semester = cell.find("a", class_="ps-link")
    semesters[semester.text] = semester.get("href")

url = semesters["Fall Semester 2024"]
driver.execute_script(url); 
time.sleep(5)

element = driver.find_element(By.ID, 'MSU_CLSRCH_WRK2_SUBJECT')  
element.send_keys("CMSE") #pick cmse for example

url = f"javascript:submitAction_win0(document.win0,'MSU_CLSRCH_WRK_SSR_PB_SEARCH');"
driver.execute_script(url); # Hit search
time.sleep(5)

In [4]:
def get_basic_info(soup):
    '''Scrape entire page for all of the class boxes'''
    # get the number of results
    result_element = soup.find('span', id='MSU_RSLT_NAV_WK_PTPG_ROWS_GRID')
    # Extract the text content
    result_text = result_element.get_text(strip=True)
    result = int(result_text.split()[0])
    pages = (result + 49) // 50 # get the number of aggregated pages
    
    reshaped_list = []
    cells = soup.find_all("tr", class_="ps_grid-row psc_rowact") # Find all table row in the page (Classes)
    if pages > 1: # If there's more than 1 page, go to the next page and scrape
        for i in range(pages - 1):
            url = f"javascript:submitAction_win0(document.win0,'MSU_RSLT_NAV_WK_SEARCH_CONDITION2$46$');"
            driver.execute_script(url)
            time.sleep(5)
            body = driver.page_source
            next_soup = BeautifulSoup(body, 'html.parser')
            cells += next_soup.find_all("tr", class_="ps_grid-row psc_rowact")
    
    for cell in cells:
        values = cell.text.split("\n") # Split text in a cell 
        values = list(filter(lambda x: x != "", values))  

        reshaped_list.append(values[:-3]) # Other columns will be scrape from SIS
    col_names = ['Course', 'Type', 'Section']
    df = pd.DataFrame(reshaped_list, columns=col_names)
    return df
        
body = driver.page_source
soup = BeautifulSoup(body, 'html.parser')
df = get_basic_info(soup)  # getting info on the first page

In [5]:
login_to_SIS(driver)

Please enter your MSU email: 


 danganh1


Please enter your password: 


 ········


Please enter your SMS code: 


 547797


In [6]:
try:
    element = driver.find_element(By.ID, "win0groupletPTNUI_LAND_REC_GROUPLET$1") # Click on Classes
    element.click()
    time.sleep(5)
except: # In case the website want to login again
    element = driver.find_element(By.ID, 'loginUrl1') # Click on Login
    element.click()
    time.sleep(5)
    
    element = driver.find_element(By.ID, "win0groupletPTNUI_LAND_REC_GROUPLET$1")  # Click on Classes
    element.click()
    time.sleep(5)

try:
    element = driver.find_element(By.ID, 'SCC_LO_FL_WRK_SCC_VIEW_BTN$2')  # Click on Class Search & Enroll
    element.click()
    time.sleep(5)
except:
    pass # If there's no Class Search & Enroll, proceed

body = driver.page_source
soup = BeautifulSoup(body, 'html.parser')
cells = soup.find_all("tr", class_="ps_grid-row psc_rowact")
semesters = dict()
for cell in cells: # Find each semester's ID
    semester = cell.find("a", class_="ps-link")
    semesters[semester.text] = semester.get("href")

url = semesters["Fall Semester 2024"]
driver.execute_script(url);
time.sleep(5)

element = driver.find_element(By.ID, 'MSU_CLSRCH_WRK2_SUBJECT')  
element.send_keys("CMSE") #pick cmse for example

url = f"javascript:submitAction_win9(document.win9,'MSU_CLSRCH_WRK_SSR_PB_SEARCH');"
driver.execute_script(url); # Hit search
time.sleep(5)

Function to get basic classes' info

In [16]:
body = driver.page_source
soup = BeautifulSoup(body, 'html.parser')
result_element = soup.find('span', id='MSU_RSLT_NAV_WK_PTPG_ROWS_GRID')
# Extract the text content
result_text = result_element.get_text(strip=True)
result = int(result_text.split()[0])
pages = (result + 49) // 50 # get the number of aggregated pages

def get_advanced_info(soup):
    schedule_ls = []
    for i in range(3): # Find the class schedules, there might be multiple
        div = soup.find("div", id=f"win9divMSU_CLS_DTL_WK2_HTMLAREA1$160$${i}")
        if not div:
            break
        schedule_ls.append(div)
        
    date_tds = soup.find_all("td", class_="ps_grid-cell E_HTMLAREA2") # Find class dates
    loc_ins_tds = soup.find_all("td", class_="ps_grid-cell E_HTMLAREA3") # Find class locations, instructors, and modes
    
    schedules = []
    dates = []
    locs = []
    modes = []
    names = []
    emails = []
    for i in range(len(loc_ins_tds)):
        # Append multiple schedules to a list
        schedules.append(schedule_ls[i].text.strip())
        
        # Append multiple dates to a list
        dates.append(date_tds[i].text.strip())

        # Append multiple locations and modes to lists
        txt = loc_ins_tds[i].get_text(strip=True, separator='\n')
        locs.append(txt.split('\n')[0])
        modes.append(txt.split('\n')[-1])

        # Append multiple professors in different schedules to lists
        a_elements = loc_ins_tds[i].find_all("a")
        if not a_elements:
            continue
        email_ind = ''
        name_ind = ''
        for a_element in a_elements: # If one schedule of the class has more than 1 professor, separate with '\n'
            # Extract the email address from the href attribute
            email_ind += a_element['href'].split(':')[1] + '\n'
            name_ind += a_element.text + '\n'
        emails.append(email_ind.strip())
        names.append(name_ind.strip())

    element = driver.find_element(By.ID, "MSU_CLS_DTL_WK2_SSR_CL_DTLS_LFF$5$_LBL") 
    element.click() # Click on Availability 
    time.sleep(2)

    avails = []
    for i in range(10):
        try: 
            element = driver.find_element(By.ID, f"win9divMSU_CLS_DTL_WK2_HTMLAREA6$22$${i}")
            avails.append(element.text)
        except: 
            break
    
    if len(avails) > 1:
        enrolled = avails.pop()
        limit = sum([int(x.split('/')[-1]) for x in avails])
        avails = [enrolled + '/' + str(limit) + '*']
    
    return schedules, dates, locs, modes, names, emails, avails[0] 

In [17]:
def add_info(df):
    '''Function to add info from the breakout windows for each course
    '''
    schedules = []
    dates = []
    locations = []
    modes = []
    emails = []
    names = []
    avails = []
    for i in range(pages): # Go through all pages
        for i in range(50): # Go through all cells in a page
            try:
                rowname = f"DESCR100$0_row_{i}"
                element = driver.find_element(By.ID, rowname) 
                print(rowname)
            except: # End of dataframe, if cannot found, break the loop
                break
            driver.execute_script("arguments[0].click();", element) # Click on cell
            time.sleep(3) # If too slow can change this line, if there's an error, increase the time
            driver.switch_to.frame(0) # Switch to Class Description frame
            body = driver.page_source
            soup = BeautifulSoup(body, 'html.parser')
            schedule, date, loc, mode, name, email, avail = get_advanced_info(soup)
            schedules.append(schedule)
            dates.append(date)
            locations.append(loc)
            modes.append(mode)
            emails.append(email)
            names.append(name)
            avails.append(avail)
            cancel_cmd="javascript:doUpdateParent(document.win9,'#ICCancel');" 
            driver.execute_script(cancel_cmd); # Close the frame
            driver.switch_to.default_content(); # Switch to the main page
            time.sleep(2)
        
        if pages-1 != i: # If there's more pages, click Next
            url = f"javascript:submitAction_win9(document.win9,'MSU_RSLT_NAV_WK_SEARCH_CONDITION2$46$');"
            driver.execute_script(url)
            time.sleep(5)
        else: # If there's no more page, break out of the big loop
            break
    
    df['Schedule'] = schedules
    df['Dates'] = dates
    df['Location'] = locations
    df['Mode'] = modes
    df['email'] = emails
    df['Instructor'] = names
    df['Availability'] = avails
    return df

In [18]:
df = add_info(df)
df.to_csv("Fall2024.csv", index=False)

DESCR100$0_row_0
DESCR100$0_row_1
DESCR100$0_row_2
DESCR100$0_row_3
DESCR100$0_row_4
DESCR100$0_row_5
DESCR100$0_row_6
DESCR100$0_row_7
DESCR100$0_row_8
DESCR100$0_row_9
DESCR100$0_row_10
DESCR100$0_row_11
DESCR100$0_row_12
DESCR100$0_row_13
DESCR100$0_row_14
DESCR100$0_row_15
DESCR100$0_row_16
DESCR100$0_row_17
DESCR100$0_row_18
DESCR100$0_row_19
DESCR100$0_row_20
DESCR100$0_row_21
DESCR100$0_row_22
DESCR100$0_row_23
DESCR100$0_row_24
DESCR100$0_row_25
DESCR100$0_row_26
DESCR100$0_row_27
DESCR100$0_row_28
DESCR100$0_row_29
DESCR100$0_row_30
DESCR100$0_row_31
DESCR100$0_row_32
DESCR100$0_row_33
DESCR100$0_row_34
DESCR100$0_row_35
DESCR100$0_row_36
DESCR100$0_row_37
DESCR100$0_row_38
DESCR100$0_row_39
DESCR100$0_row_40
DESCR100$0_row_41
DESCR100$0_row_42
DESCR100$0_row_43
DESCR100$0_row_44
DESCR100$0_row_45
DESCR100$0_row_46
DESCR100$0_row_47
DESCR100$0_row_48
DESCR100$0_row_49
DESCR100$0_row_0
DESCR100$0_row_1
DESCR100$0_row_2
DESCR100$0_row_3
DESCR100$0_row_4
DESCR100$0_row_5
DESCR100

In [19]:
def make_courses_df(df): 
    # Explode all these columns, will result in duplicate rows for classes that have multiple schedules
    df = df.explode(['Schedule', 'Dates', 'Location', 'Mode', 'email', 'Instructor'])
            
    df[['Days', 'Time']] = df['Schedule'].str.split(':',n=1,expand=True)
    df[['Course Code', 'Course Name']] = df['Course'].str.split(':', n=1, expand=True)  
    split_result = df['Type'].str.split('(', n=1, expand=True)
    # Check if the split operation resulted in two columns
    if len(split_result.columns) == 2:
        df[['Type', 'Units']] = split_result
    else:
        # Handle the case where the split didn't result in two columns
        df['Type'] = split_result[0]  # Assign the first part to 'Type'
        df['Units'] = '' 
    df[['Section', 'Class Nbr', 'Academic Session']] = df['Section'].str.split('/', n=2, expand=True)
    df[['Units','Status']] = df['Units'].str.split(')',n=1,expand=True)
    df[['Subject','Course Number']] = df['Course Code'].str.split(' ',n=1,expand=True)
    df['Dates'] = df['Dates'].apply(lambda x: x.replace("Approval Required", '').strip() if not pd.isna(x) else x)
    df['Status'] = df['Status'].str.replace('Reserved Capacity', '').str.strip()
    
    #df['Dates'] = df['Dates'].str.replace('Approval Required', '').str.strip()
    df['Course Name'] = df['Course Name'].str.replace('Cross-Listed', '').str.strip()
    df['Course Name'] = df['Course Name'].str.replace('Approval Required', '').str.strip()

    df = df.drop(['Course', 'Schedule','Course Code'], axis=1)
    df = df[['Subject','Course Number','Course Name','Type','Units','Status',
             'Section','Class Nbr','Academic Session','Days','Time','Dates', 
             'Location', 'Mode', 'email', 'Instructor', 'Availability']]
    df['Units'] = df['Units'].str.replace(' units', '')
    df['Section'] = df['Section'].str.extract(r'(\d+(?:\.\d+)?)')
    df['Class Nbr'] = df['Class Nbr'].str.extract(r'(\d+(?:\.\d+)?)')

    df.replace(['Instructor:', 'To Be Announced', 'Arranged'], None, inplace=True)
    return df

In [20]:
df = make_courses_df(df)
df

,Subject,Course Number,Course Name,Type,Units,Status,Section,Class Nbr,Academic Session,Days,Time,Dates,Location,Mode,email,Instructor,Availability
0,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,001,14121,Regular Academic Session,Mon Wed,8:00 AM-9:50 AM,1/8/2024 - 4/21/2024,STEM 3201,In Person,baolianz@msu.edu,Lianzhang Bao,57/60
1,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,002,19656,Regular Academic Session,Mon Wed,10:20 AM-12:10 PM,1/8/2024 - 4/21/2024,STEM 3201,In Person,baolianz@msu.edu,Lianzhang Bao,57/60
2,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,003,14122,Regular Academic Session,Mon Wed,12:40 PM-2:30 PM,1/8/2024 - 4/21/2024,STEM 3201,In Person,silves28@msu.edu,Luciano Silvestri,55/60
3,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,004,14123,Regular Academic Session,Mon Wed,3:00 PM-4:50 PM,1/8/2024 - 4/21/2024,STEM 3201,In Person,alexrd@msu.edu,Alexander Dickson,59/60
4,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,005,14124,Regular Academic Session,Tue Thu,8:00 AM-9:50 AM,1/8/2024 - 4/21/2024,STEM 3201,In Person,mengsen@msu.edu,Mengsen Zhang,55/60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,CMSE,891,Independent Study in Computational Mathematics...,Lecture,1.00 - 4.00,Open,005,37761,Regular Academic Session,None,None,1/8/2024 - 4/21/2024,None,In Person,christli@msu.edu,Andrew Christlieb,2/6
58,CMSE,891,Independent Study in Computational Mathematics...,Lecture,3.00,Wait List,006,37838,Regular Academic Session,None,None,1/8/2024 - 4/21/2024,None,In Person,salmonra@msu.edu,Rachel Frisbie,1/1
59,CMSE,891,Independent Study in Computational Mathematics...,Lecture,3.00,Wait List,007,37839,Regular Academic Session,None,None,1/8/2024 - 4/21/2024,None,In Person,salmonra@msu.edu,Rachel Frisbie,1/1
60,CMSE,899,Master's Thesis Research,Lecture,1.00 - 6.00,Open,001,17519,Regular Academic Session,None,None,1/8/2024 - 4/21/2024,None,In Person,NaN,NaN,0/1


In [21]:
df.to_csv('Fall2024.csv')